In [14]:
with open('marvel.txt','r',encoding='utf-8') as f:
    text = f.read()

In [15]:
print(text[:600])

I still think about the night
your mother and I had to leave you.
Hopefully, it's not for long,
but I'll call you when we get settled.
When I have a better indication of what's going on.
Mommy?
Jellybean.
Daddy and I have a last-minute business trip...
so Rose is gonna stay with you for a few days.
No. I don't want you to go.
Ugh. It's gonna be so boring.
I won't be able to keep my...
eyes...
Goodbye, sweetheart. All right, we'll see you soon.
Okay.
Janet, we gotta go.
I wish we could have put down our bags...
tucked you back into your bed...
but too many lives were at stake.
Oh, my God.
They'


In [16]:
len(text)

1147324

In [17]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)
print([ord(char) for char in chars])


 !"$%&'()*,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
80
[10, 32, 33, 34, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 63, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122]


In [18]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[ch] for ch in s]
decode = lambda lst: ''.join([itos[i] for i in lst])

In [19]:
import torch
data = torch.tensor(encode(text),dtype=torch.long,device='cuda')
print(decode(data[:100].tolist()))
device = 'cuda' if torch.cuda.is_available() else 'cpu'

I still think about the night
your mother and I had to leave you.
Hopefully, it's not for long,
but 


In [20]:
train_len = int(0.9*len(data))
train_data = data[:train_len]
val_data = data[train_len:]
print(decode(train_data[:100].tolist()))

I still think about the night
your mother and I had to leave you.
Hopefully, it's not for long,
but 


In [21]:
block_size = 8
batch_size = 4
n_embd = 64

In [22]:
def get_data(splittype):
    data = val_data
    if(splittype=='train'): 
        data = train_data
    ix = torch.randint(len(data)-block_size,(batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    return x,y

xb, yb = get_data('train')
print(xb)
print(yb)

tensor([[57, 67,  7, 73,  1, 61, 54, 69],
        [62, 60, 61, 73,  1, 72, 58, 58],
        [72,  1, 69, 74, 73,  1, 68, 67],
        [76, 62, 73, 61,  1, 39, 68, 64]], device='cuda:0')
tensor([[67,  7, 73,  1, 61, 54, 69, 69],
        [60, 61, 73,  1, 72, 58, 58,  1],
        [ 1, 69, 74, 73,  1, 68, 67,  1],
        [62, 73, 61,  1, 39, 68, 64, 62]], device='cuda:0')


In [23]:
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b,t]
        print(f"{decode(context.tolist())}->{decode([target.item()])}")

d->n
dn->'
dn'->t
dn't-> 
dn't ->h
dn't h->a
dn't ha->p
dn't hap->p
i->g
ig->h
igh->t
ight-> 
ight ->s
ight s->e
ight se->e
ight see-> 
s-> 
s ->p
s p->u
s pu->t
s put-> 
s put ->o
s put o->n
s put on-> 
w->i
wi->t
wit->h
with-> 
with ->L
with L->o
with Lo->k
with Lok->i


In [25]:
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd,device=device) 
        self.positional_embedding_table = nn.Embedding(block_size,n_embd,device=device)
        self.lm_head = nn.Linear(n_embd, vocab_size,device=device)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)  # B x T x C
        pos_emb = self.positional_embedding_table(torch.arange(T,device=device))
        x = tok_emb + pos_emb
        logits = self.lm_head(x)  # B x T x vocab_size
        if targets is None:
            loss = None
        else:
            C = logits.shape[-1]
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits,loss = self(idx[:,-block_size:])
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

model = BigramLanguageModel()
out = model(xb,yb)
print(decode(model.generate(torch.zeros((1,1),device=device,dtype=torch.long),max_new_tokens=100)[0].tolist()))


o-&!v2FG'1
0BBBXQST8-sT"xj8ucc(SSSiJ-5,DjJV8L'5.M1$f$H)SBDQuNJBJ8lB?SSj)BeCvPztByz Kf.Sj.$x";!OQrdIS


In [26]:
optimizer = torch.optim.AdamW(model.parameters(),lr=1e-3)

In [30]:
batch_size = 32
for steps in range(30000):
    xb,yb = get_data('train')
    logits,loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

In [31]:
context = torch.zeros((1,1),dtype=torch.long,device=device)
print(decode(model.generate(context,max_new_tokens=500)[0].tolist()))


whin sangot's g houm fu t.
Ge hareaprey mo ad undow of TEve.
Wan..
TEar wime'torry s Plyo mfilitana s mesp ll re in ongong?
- is bokerr cateduckigugus iversed.
Toupplove?
Une. ou.
Youn..
H l..
S: atoureanl wowee ad heinear Gomes.
Simy fiaceye s o e p orengos y. g banope.. wasan guristh..
Heve g w t tharer St'se worenche hastun the Dit hene tr- l tu teak gal m treat m.
Year anghanbore p.
Yoiecr hecanat urug smunathout n ofotann. s the y s soffirind! an I INGRURanas...
Stho I w s yofominge..
Averm


True